<a href="https://colab.research.google.com/github/DHarley22/Prediction_case_malaria_mozambique/blob/1-xgboost-predictor-python-version/model_building_mozambique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

%%capture
!pip install gdown optuna


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import numpy as np
import xgboost as xgb
import sklearn
import gdown
import optuna
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as split_into_two
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from xgboost import XGBRegressor

In [ ]:
%%capture

# The mozambique dataset is hosted here
# https://drive.google.com/file/d/1uHGOjxlxfDYY5E-aL9HVRGvheDa_wNyq/view?usp=sharing
dataset_drive_id = '1uHGOjxlxfDYY5E-aL9HVRGvheDa_wNyq'
dataset_output_file = 'downloaded_sheet.xlsx'

# The shape file relatd to the dataset is hosted here
# https://drive.google.com/drive/folders/14UJ7ZXWmNeL28sYAv6dObNsC42kQr4Ja?usp=sharing
shape_file_drive_id = '14UJ7ZXWmNeL28sYAv6dObNsC42kQr4Ja'
shape_output_folder = 'shape_files'

gdown.download_folder(id=shape_file_drive_id, output=shape_output_folder, quiet=False)
gdown.download(id=dataset_drive_id, output=dataset_output_file, quiet=False)


FileURLRetrievalError: Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1lSLOnObKtgGV6IIvFEndLu5U3voslar5

but Gdown can't. Please check connections and permissions.

In [ ]:
dataset = pd.read_csv(f"/content/{dataset_output_file}")
shape_file = gpd.read_file(f"/content/{shape_output_folder}/shape_file.shp")

In [ ]:
# Identify districts where malaria_cases_u5 and diarr_cases_u5 have no values strictly greater than 0
districts_to_remove = []
for district in dataset['district'].unique():
    district_data = dataset[dataset['district'] == district]

    # Check if all values in 'malaria_cases_u5' and 'diarr_cases_u5' are ≤ 0 or NaN
    if ((district_data['malaria_cases_u5'] <= 0) | district_data['malaria_cases_u5'].isna()).all() and \
       ((district_data['diarr_cases_u5'] <= 0) | district_data['diarr_cases_u5'].isna()).all():
        districts_to_remove.append(district)


print("districts_to_remove => ", districts_to_remove)



districts_to_remove =>  ['cidade da matola', 'cidade de chimoio', 'cidade de inhambane', 'cidade de tete', 'ibo', 'ilha licom', 'ilha risunodo', 'lago niassa', 'maquival', 'maxixe', 'nacala-a-velha']


In [ ]:
# Remove the districts
dataset = dataset[~dataset['district'].isin(districts_to_remove)]

In [ ]:
# (Left) join the dataset to the shape_file using the "district" field
shaped_dataset = dataset.merge(
    shape_file[["district", "geometry"]],
    on="district",
    how="left"
)

# Convert the Dataframe into GeoDataframe and add some fields to it
shaped_dataset =  gpd.GeoDataFrame(shaped_dataset, geometry="geometry")

shaped_dataset = shaped_dataset.to_crs(epsg=3857)

# Calculate the area and centroid using the projected CRS
shaped_dataset["area"] = shaped_dataset.geometry.area
shaped_dataset["centroid_x"] = shaped_dataset.geometry.centroid.x
shaped_dataset["centroid_y"] = shaped_dataset.geometry.centroid.y

In [ ]:
shaped_dataset[shaped_dataset['district'] == "angonia"]

,Unnamed: 0,district,year,month,district_SPH,malaria_cases_u5,diarr_cases_u5,population,tmin,tmax,...,prop_dwelling_sprayed_last_12_Months,prop_with_3Plus_mosquito_nets,prop_children_under_mosquito_bed_nets_previous_night2Plus,prop_uneducated,number_of_doctors,Name_of_healthcare_facility1,geometry,area,centroid_x,centroid_y
108,109,angonia,2016,1,angonia,8549.0,860.0,357186.292306,19.071238,25.384994,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06
109,110,angonia,2016,2,angonia,4309.0,455.0,357186.292306,18.638211,24.039289,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06
110,111,angonia,2016,3,angonia,7504.0,339.0,357186.292306,18.946583,24.210317,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06
111,112,angonia,2016,4,angonia,5142.0,219.0,357186.292306,16.281213,21.651856,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06
112,113,angonia,2016,5,angonia,4245.0,177.0,357186.292306,13.066037,19.529904,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06
113,114,angonia,2016,6,angonia,3882.0,307.0,357186.292306,11.354505,17.686322,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06
114,115,angonia,2016,7,angonia,2136.0,250.0,357186.292306,12.641906,18.782725,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06
115,116,angonia,2016,8,angonia,1408.0,285.0,357186.292306,13.064437,20.117401,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06
116,117,angonia,2016,9,angonia,1837.0,300.0,357186.292306,15.010347,23.058529,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06
117,118,angonia,2016,10,angonia,785.0,107.0,357186.292306,17.729661,27.247561,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06


In [ ]:
# Group the data by district
grouped = shaped_dataset.groupby('district')

# Iterate through each district
for district, group_data in grouped:
    # Compute the mean, ignoring both 0s and NaNs
    malaria_mean = group_data.loc[
        (group_data['malaria_cases_u5'] > 0) & (~group_data['malaria_cases_u5'].isna()),
        'malaria_cases_u5'
    ].mean()

    diarr_mean = group_data.loc[
        (group_data['diarr_cases_u5'] > 0) & (~group_data['diarr_cases_u5'].isna()),
        'diarr_cases_u5'
    ].mean()

    # Fill only 0 and NaN values
    shaped_dataset.loc[
        (shaped_dataset['district'] == district) &
        ((shaped_dataset['malaria_cases_u5'] == 0) | (shaped_dataset['malaria_cases_u5'].isna())),  # Condition for 0 or NaN
        'malaria_cases_u5'
    ] = malaria_mean

    shaped_dataset.loc[
        (shaped_dataset['district'] == district) &
        ((shaped_dataset['diarr_cases_u5'] == 0) | (shaped_dataset['diarr_cases_u5'].isna())),  # Condition for 0 or NaN
        'diarr_cases_u5'
    ] = diarr_mean


======================== District alto molocue ========================
malaria_mean =>  4530.0344827586205
diarr_mean =>  173.86206896551724
======================== District ancuabe ========================
malaria_mean =>  2246.53125
diarr_mean =>  149.90625
======================== District angoche ========================
malaria_mean =>  3197.15625
diarr_mean =>  134.90625
======================== District angonia ========================
malaria_mean =>  3484.5666666666666
diarr_mean =>  217.93333333333334
======================== District balama ========================
malaria_mean =>  2633.28125
diarr_mean =>  25.40625
======================== District barue ========================
malaria_mean =>  2468.483870967742
diarr_mean =>  127.93548387096774
======================== District bilene ========================
malaria_mean =>  1045.9032258064517
diarr_mean =>  78.51612903225806
======================== District boane ========================
malaria_mean =>  139.19354838

In [ ]:
shaped_dataset[shaped_dataset['district'] == "angonia"]

,Unnamed: 0,district,year,month,district_SPH,malaria_cases_u5,diarr_cases_u5,population,tmin,tmax,...,prop_dwelling_sprayed_last_12_Months,prop_with_3Plus_mosquito_nets,prop_children_under_mosquito_bed_nets_previous_night2Plus,prop_uneducated,number_of_doctors,Name_of_healthcare_facility1,geometry,area,centroid_x,centroid_y
108,109,angonia,2016,1,angonia,8549.000000,860.000000,357186.292306,19.071238,25.384994,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06
109,110,angonia,2016,2,angonia,4309.000000,455.000000,357186.292306,18.638211,24.039289,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06
110,111,angonia,2016,3,angonia,7504.000000,339.000000,357186.292306,18.946583,24.210317,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06
111,112,angonia,2016,4,angonia,5142.000000,219.000000,357186.292306,16.281213,21.651856,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06
112,113,angonia,2016,5,angonia,4245.000000,177.000000,357186.292306,13.066037,19.529904,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06
113,114,angonia,2016,6,angonia,3882.000000,307.000000,357186.292306,11.354505,17.686322,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06
114,115,angonia,2016,7,angonia,2136.000000,250.000000,357186.292306,12.641906,18.782725,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06
115,116,angonia,2016,8,angonia,1408.000000,285.000000,357186.292306,13.064437,20.117401,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06
116,117,angonia,2016,9,angonia,1837.000000,300.000000,357186.292306,15.010347,23.058529,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06
117,118,angonia,2016,10,angonia,785.000000,107.000000,357186.292306,17.729661,27.247561,...,0.0,15.32,3.05,28.12,2103.0,15.0,"POLYGON ((3824882.183 -1618561.805, 3825042.70...",3.504505e+09,3.802997e+06,-1.651805e+06


### Missing data handling

Rules:

- If {some condition} then {we do this}
- If {some condition} then {we do this}

In [ ]:
# Handle missing data
# TODO:

### Test/Val/Test Split

Rules:

- We will use


In [ ]:
TARGET_DISEASE_FIELD = "malaria_cases_u5"


# Relevant features
relevant_features = [
    # Environemental features
    'tmin', 'tmax', 'precipitation', 'ndvi','RH',
    # Socio-economic features
    'prop_poor', 'prop_Rural', 'prop_drinking_TreatedWater',
    # Spatial features
     'centroid_x', 'centroid_y', 'area'
]

# Test dataset predicate; use lines between March and June for testing
test_data_predicate = (shaped_dataset['year'] == 2018) & (shaped_dataset['month'].between(3, 6))

# Test dataset
X_test = shaped_dataset[test_data_predicate][relevant_features]
y_test = shaped_dataset[test_data_predicate][TARGET_DISEASE_FIELD]

# The rest of the dataset separated as X, y
X = shaped_dataset[~test_data_predicate][relevant_features]
y = shaped_dataset[~test_data_predicate][TARGET_DISEASE_FIELD]

print(X.shape, y.shape)
X_train, X_val, y_train, y_val = split_into_two(X, y, test_size=0.3, random_state=42)




(4736, 11) (4736,)


In [ ]:
assert False, "Please don't run this unless you want to scan the parameter space again. We've got a better parameter that can be used to same time. The objective function call has been commented"


# Define the objective function for Optuna
def objective(trial):
    # Hyperparameter search space
    params = {
        'verbosity': 0,
        'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
        'objective': 'reg:squarederror',  # Regression objective
        'eval_metric': 'rmse',            # Root Mean Squared Error
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 2, 30),
        'gamma': trial.suggest_float('gamma', 0.01, 10, log=True),
        'subsample': trial.suggest_float('subsample', 0.01, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-7, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-7, 1.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 0, 10),
        'n_estimators': trial.suggest_int('n_estimators', 5, 5000),
    }


    # Cross-validation
    cv_results = xgb.cv(
        params=params,
        dtrain=xgb.DMatrix(X_train, label=y_train),
        nfold=5,
        metrics='rmse',
        num_boost_round=params['n_estimators'],
        as_pandas=True,
        verbose_eval=50,
        seed=0,
        early_stopping_rounds=20
    )

    return cv_results['test-rmse-mean'].iloc[-1]

# Run Optuna optimization (minization problem)
# study = optuna.create_study(study_name="xgboost_model_study", direction="minimize")
# study.optimize(objective, n_trials=25)

# Best hyperparameters
print("Best hyperparameters:", study.best_params)


[I 2025-01-29 21:17:47,779] A new study created in memory with name: xgboost_model_study


[0]	train-rmse:1820.81190+13.41626	test-rmse:1820.45964+54.06944
[50]	train-rmse:1818.74266+13.36879	test-rmse:1818.42042+54.04815
[100]	train-rmse:1816.66935+13.32813	test-rmse:1816.38441+54.00737
[150]	train-rmse:1814.60256+13.27870	test-rmse:1814.34261+53.99382
[200]	train-rmse:1812.52405+13.21889	test-rmse:1812.29614+53.99017
[250]	train-rmse:1810.54712+13.18741	test-rmse:1810.35405+53.97505
[300]	train-rmse:1808.52599+13.13854	test-rmse:1808.35781+53.95907
[350]	train-rmse:1806.50654+13.08422	test-rmse:1806.38715+53.95210
[400]	train-rmse:1804.46613+13.02395	test-rmse:1804.39021+53.94554
[450]	train-rmse:1802.41660+12.97012	test-rmse:1802.37318+53.93916
[500]	train-rmse:1800.44054+12.90675	test-rmse:1800.43806+53.95473
[550]	train-rmse:1798.43600+12.85665	test-rmse:1798.48224+53.93504
[600]	train-rmse:1796.47887+12.80955	test-rmse:1796.56816+53.92681
[650]	train-rmse:1794.46063+12.76568	test-rmse:1794.58163+53.91422
[700]	train-rmse:1792.44666+12.71646	test-rmse:1792.59969+53.9099

[I 2025-01-29 21:18:07,597] Trial 0 finished with value: 1724.6988286505962 and parameters: {'booster': 'gbtree', 'learning_rate': 4.5291539119870224e-05, 'max_depth': 4, 'gamma': 0.6614781664058753, 'subsample': 0.5118864734120295, 'reg_alpha': 1.1354454185869866e-06, 'reg_lambda': 0.31225904145169037, 'colsample_bytree': 0.5522072061997629, 'min_child_weight': 5, 'n_estimators': 2526}. Best is trial 0 with value: 1724.6988286505962.


[0]	train-rmse:1820.38517+13.41079	test-rmse:1820.06398+54.08705
[50]	train-rmse:1797.79478+13.11439	test-rmse:1799.05010+54.30378
[100]	train-rmse:1775.58140+13.02370	test-rmse:1778.36933+54.40747
[150]	train-rmse:1753.84940+12.80415	test-rmse:1758.23801+54.67133
[200]	train-rmse:1732.18352+12.55177	test-rmse:1738.14385+55.12088
[250]	train-rmse:1711.34513+12.31027	test-rmse:1719.05257+55.59237
[300]	train-rmse:1690.55222+12.11466	test-rmse:1699.90237+55.93035
[350]	train-rmse:1670.25799+11.88670	test-rmse:1681.30776+56.37107
[400]	train-rmse:1649.97567+11.66601	test-rmse:1662.62287+56.87640
[450]	train-rmse:1629.96378+11.44613	test-rmse:1644.19964+57.47901
[500]	train-rmse:1610.41621+11.29728	test-rmse:1626.50334+58.09569
[550]	train-rmse:1591.23869+11.11426	test-rmse:1609.04927+58.61052
[600]	train-rmse:1572.63586+10.95326	test-rmse:1592.27979+59.11748
[650]	train-rmse:1553.92849+10.80097	test-rmse:1575.29191+59.57935
[700]	train-rmse:1535.61252+10.63809	test-rmse:1558.68736+60.1243

[I 2025-01-29 21:22:03,927] Trial 1 finished with value: 928.5332206108021 and parameters: {'booster': 'gbtree', 'learning_rate': 0.00030918437864959844, 'max_depth': 24, 'gamma': 0.2530290350717577, 'subsample': 0.5854261700508007, 'reg_alpha': 0.0001740042716208815, 'reg_lambda': 4.3117607124943137e-07, 'colsample_bytree': 0.6116695784094005, 'min_child_weight': 5, 'n_estimators': 4471}. Best is trial 1 with value: 928.5332206108021.


[0]	train-rmse:1820.81143+13.41662	test-rmse:1820.46002+54.07073
[50]	train-rmse:1818.61377+13.39303	test-rmse:1818.38565+54.10524
[100]	train-rmse:1816.41536+13.37633	test-rmse:1816.30710+54.13370
[150]	train-rmse:1814.22735+13.35065	test-rmse:1814.23725+54.16938
[200]	train-rmse:1812.03465+13.32791	test-rmse:1812.16735+54.20153
[250]	train-rmse:1809.85095+13.30638	test-rmse:1810.11021+54.23666
[300]	train-rmse:1807.66837+13.28480	test-rmse:1808.04818+54.26519
[350]	train-rmse:1805.49638+13.26409	test-rmse:1805.99816+54.29472
[400]	train-rmse:1803.32262+13.24413	test-rmse:1803.94643+54.32519
[450]	train-rmse:1801.14577+13.22272	test-rmse:1801.88685+54.36167
[500]	train-rmse:1798.97802+13.20915	test-rmse:1799.84122+54.38905
[550]	train-rmse:1796.81618+13.18839	test-rmse:1797.80424+54.42751
[600]	train-rmse:1794.65149+13.16109	test-rmse:1795.76000+54.46768
[650]	train-rmse:1792.49085+13.13469	test-rmse:1793.71885+54.51387
[700]	train-rmse:1790.33590+13.11574	test-rmse:1791.69343+54.5467

[I 2025-01-29 21:23:12,869] Trial 2 finished with value: 1769.38582189113 and parameters: {'booster': 'gbtree', 'learning_rate': 2.9337851105111112e-05, 'max_depth': 14, 'gamma': 0.047204138256284646, 'subsample': 0.7148010358831169, 'reg_alpha': 1.736794100777892e-05, 'reg_lambda': 5.898552604131132e-05, 'colsample_bytree': 0.8751967632311815, 'min_child_weight': 7, 'n_estimators': 1256}. Best is trial 1 with value: 928.5332206108021.


[0]	train-rmse:1820.37353+13.40964	test-rmse:1820.04073+54.07281
[50]	train-rmse:1797.68032+13.29457	test-rmse:1798.67991+54.03648
[100]	train-rmse:1775.05606+13.21368	test-rmse:1777.30071+54.25191
[150]	train-rmse:1753.19441+13.47197	test-rmse:1756.64868+53.94332
[200]	train-rmse:1731.94352+13.56164	test-rmse:1736.74308+53.85726
[250]	train-rmse:1710.96373+13.62861	test-rmse:1716.95773+53.97005
[300]	train-rmse:1690.38411+13.56883	test-rmse:1697.62697+54.22841
[350]	train-rmse:1670.04063+13.34944	test-rmse:1678.50201+54.56173
[400]	train-rmse:1650.02537+13.05808	test-rmse:1659.57446+55.11458
[450]	train-rmse:1630.59942+13.12784	test-rmse:1641.37896+55.06487


In [ ]:
# This is the best params we got in one of our past run
best_params = {'booster': 'gbtree', 'learning_rate': 0.0035209961607260945, 'max_depth': 9, 'gamma': 0.009596716772915171, 'subsample': 0.709187029135732, 'reg_alpha': 1.6638871630185252e-07, 'reg_lambda': 0.1951894062259646, 'colsample_bytree': 0.816001373149942, 'min_child_weight': 7, 'n_estimators': 3041}

In [ ]:
# Here we will use the best parameters to train the model
d_matrix_train = xgb.DMatrix(X_train, label=y_train)
d_matrix_val = xgb.DMatrix(X_val, label=y_val)
d_matrix_test = xgb.DMatrix(X_test, label=y_test)
best_model = xgb.train(
    best_params,
    xgb.DMatrix(X_train, label=y_train),
    num_boost_round=study.best_params["n_estimators"],
    evals=[(d_matrix_train, 'train'), (d_matrix_val, 'val')],
    verbose_eval=50
)
